## 1. Predicción de Aprobaciones de Tarjetas de Crédito

Usaremos el conjunto de datos de aprobación de tarjetas de crédito del repositorio de aprendizaje automático de [UCI](https://archive.ics.uci.edu/ml/datasets/Credit+Approval). La estructura de este notebook es la siguiente:

- Primero, comenzaremos cargando y viendo el conjunto de datos.
- Veremos que el conjunto de datos tiene una mezcla de características numéricas y no numéricas, que contiene valores de diferentes rangos, además de que contiene una cantidad de datos faltantes.
- Tendremos que pre-procesar el conjunto de datos para asegurarnos de que el modelo de aprendizaje automático que elegimos pueda realizar buenas predicciones.
- Una vez que nuestros datos estén en buena forma, haremos un análisis de datos exploratorio para construir nuestras intuiciones.
- Finalmente, crearemos un modelo de aprendizaje automático que puede predecir si se aceptará la solicitud de una persona para una tarjeta de crédito.

Primero, cargar y ver el conjunto de datos. Descubrimos que, dado que estos datos son confidenciales, el contribuidor del conjunto de datos ha anonimizado los nombres de muchas de las características.

In [2]:
# Import pandas
import pandas as pd

# Load dataset
cc_apps = pd.read_table("https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data", header=None, delimiter = ',')

# Inspect data
print(cc_apps.head(17))

   0      1       2  3  4   5  6      7  8  9   10 11 12     13     14 15
0   b  30.83   0.000  u  g   w  v  1.250  t  t   1  f  g  00202      0  +
1   a  58.67   4.460  u  g   q  h  3.040  t  t   6  f  g  00043    560  +
2   a  24.50   0.500  u  g   q  h  1.500  t  f   0  f  g  00280    824  +
3   b  27.83   1.540  u  g   w  v  3.750  t  t   5  t  g  00100      3  +
4   b  20.17   5.625  u  g   w  v  1.710  t  f   0  f  s  00120      0  +
5   b  32.08   4.000  u  g   m  v  2.500  t  f   0  t  g  00360      0  +
6   b  33.17   1.040  u  g   r  h  6.500  t  f   0  t  g  00164  31285  +
7   a  22.92  11.585  u  g  cc  v  0.040  t  f   0  f  g  00080   1349  +
8   b  54.42   0.500  y  p   k  h  3.960  t  f   0  f  g  00180    314  +
9   b  42.50   4.915  y  p   w  v  3.165  t  f   0  t  g  00052   1442  +
10  b  22.08   0.830  u  g   c  h  2.165  f  f   0  t  g  00128      0  +
11  b  29.92   1.835  u  g   c  h  4.335  t  f   0  f  g  00260    200  +
12  a  38.25   6.000  u  g   k  v  1.0

## 2. Inspeccionar data

El resultado puede parecer un poco confuso a primera vista, pero intentemos descubrir las características más importantes de una aplicación de tarjeta de crédito. Las características de este conjunto de datos se han anonimizado para proteger la privacidad. Las características probables en una solicitud de tarjeta de crédito típica son:

- Género
- Edad
- Deuda
- Casado
- Cliente bancario
- Nivel de educación
- Etnia
- Años de empleo
- Predeterminado
- Empleado
- Puntuación de crédito
- Licencia de conductor
- Ciudadano
- Código postal
- Ingresos
- Estado de aprobación.

Esto nos da un buen punto de partida y podemos mapear estas características con respecto a las columnas en la salida. Como podemos ver en nuestro primer vistazo a los datos, el conjunto de datos tiene una combinación de características numéricas y no numéricas. Esto se puede solucionar con un poco de procesamiento previo, pero antes de hacerlo, aprendamos un poco más sobre el conjunto de datos para ver si hay otros problemas del conjunto de datos que deban solucionarse.

In [20]:
# Print summary statistics
cc_apps_description = cc_apps.describe()
print(cc_apps_description)

print("\n")

# Print DataFrame information
cc_apps_info = cc_apps.info()
print(cc_apps_info)

print("\n")

# Inspect missing values in the dataset
print(cc_apps.tail(17))

               2           7          10             14
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 no

## Manejo de valores faltantes (Parte 1)

Hemos descubierto algunos problemas que afectarán el rendimiento de nuestros modelos de aprendizaje automático si no cambian: nuestro conjunto de datos contiene datos numéricos y no numéricos (específicamente datos que son de tipo float64, int64 y object). Específicamente, las características 2, 7, 10 y 14 contienen valores numéricos (de tipos float64, float64, int64 e int64 respectivamente) y todas las demás características contienen valores no numéricos. El conjunto de datos también contiene valores de varios rangos. Algunas características tienen un rango de valores de 0 a 28, algunas tienen un rango de 2 a 67 y algunas tienen un rango de 1017 a 100000. Aparte de estas, podemos obtener información estadística útil (como media, máxima y mínima) sobre las características que tienen valores numéricos. Finalmente, el conjunto de datos tiene valores faltantes, de los que nos ocuparemos en esta tarea. Los valores faltantes en el conjunto de datos están etiquetados con '?'. 

Ahora, reemplacemos temporalmente estos signos de interrogación de valores faltantes con NaN.

In [21]:
# Import numpy
import numpy as np

# Replace the '?'s with NaN
cc_apps=cc_apps.replace('?',np.NaN)

# Inspect the missing values again
print(cc_apps.tail(17))

print("\n")

#count the total number of NaNs for the whole dataframe
nans = cc_apps.isnull().values.sum()
print(nans)

      0      1       2  3  4   5   6      7  8  9   10 11 12     13   14 15
673  NaN  29.50   2.000  y  p   e   h  2.000  f  f   0  f  g  00256   17  -
674    a  37.33   2.500  u  g   i   h  0.210  f  f   0  f  g  00260  246  -
675    a  41.58   1.040  u  g  aa   v  0.665  f  f   0  f  g  00240  237  -
676    a  30.58  10.665  u  g   q   h  0.085  f  t  12  t  g  00129    3  -
677    b  19.42   7.250  u  g   m   v  0.040  f  t   1  f  g  00100    1  -
678    a  17.92  10.210  u  g  ff  ff  0.000  f  f   0  f  g  00000   50  -
679    a  20.08   1.250  u  g   c   v  0.000  f  f   0  f  g  00000    0  -
680    b  19.50   0.290  u  g   k   v  0.290  f  f   0  f  g  00280  364  -
681    b  27.83   1.000  y  p   d   h  3.000  f  f   0  f  g  00176  537  -
682    b  17.08   3.290  u  g   i   v  0.335  f  f   0  t  g  00140    2  -
683    b  36.42   0.750  y  p   d   v  0.585  f  f   0  f  g  00240    3  -
684    b  40.58   3.290  u  g   m   v  3.500  f  f   0  t  s  00400    0  -
685    b  21

## 4. Manejo de los valores faltantes (Parte 2)

Reemplazamos todos los signos de interrogación con NaN. Esto nos ayudará en el próximo tratamiento de valor perdido que realizaremos. Una pregunta importante que se plantea aquí es *¿por qué le damos tanta importancia a los valores faltantes?*, *¿No se pueden simplemente ignorar?* Ignorar los valores faltantes puede afectar en gran medida el rendimiento de un modelo de aprendizaje automático. Si ignoramos los valores faltantes, nuestro modelo de aprendizaje automático puede perder información sobre el conjunto de datos que puede ser útil para su entrenamiento. Entonces, hay muchos modelos que no pueden manejar valores perdidos implícitamente, como LDA. Entonces, para evitar este problema, vamos a imputar los valores perdidos con una estrategia llamada *imputación media*.

In [22]:
# Impute the missing values with mean imputation
cc_apps.fillna(cc_apps.mean(), inplace=True) #fillna only fills up the numeric columns

# Count the number of NaNs for each column in the dataset to verify
print(cc_apps.isnull().sum())

# Count the total number of NaNs for the whole dataframe
print(cc_apps.isnull().values.sum())


0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64
67


## 5. Manejo de los valores faltantes (Parte 3)

Nos hemos ocupado con éxito de los valores faltantes presentes en las columnas numéricas. Todavía hay algunos valores faltantes para imputar para las columnas 0, 1, 3, 4, 5, 6 y 13. Todas estas columnas contienen datos no numéricos y por eso la estrategia de imputación media no funcionaría aquí. Esto necesita un tratamiento diferente. Vamos a imputar estos valores perdidos con los valores más frecuentes presentes en las columnas respectivas. Esta es una buena práctica cuando se trata de imputar valores perdidos para datos categóricos en general.


In [23]:
# Iterate over each column of cc_apps
for col in cc_apps.columns:
    # Check if the column is of object type
    if cc_apps[col].dtypes == 'object':
        # Impute with the most frequent value
        cc_apps = cc_apps.fillna(cc_apps[col].value_counts().index[0])

# Count the number of NaNs in the dataset and print the counts to verify.
print(cc_apps.isnull().values.sum())

0


In [ ]:
## 6. Procesamiento previo de los datos (Parte 1)

Los valores faltantes ahora se manejan con éxito.

Todavía se necesita un preprocesamiento de datos menor pero esencial antes de continuar con la construcción de nuestro modelo de aprendizaje automático. Vamos a dividir estos pasos restantes de preprocesamiento en tres tareas principales:

- Convierta los datos no numéricos en numéricos.
- Divida los datos en conjuntos de prueba y de entrenamiento.
- Escale los valores a un rango uniforme.

Primero, convertiremos todos los valores no numéricos en valores numéricos. Hacemos esto porque no solo da como resultado un cálculo más rápido, sino que también muchos modelos de aprendizaje automático (como XGBoost) (y especialmente los desarrollados con scikit-learn) requieren que los datos estén en un formato estrictamente numérico. Haremos esto usando una técnica llamada *codificación de etiquetas*.

In [24]:
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder
# Instantiate LabelEncoder
le = LabelEncoder()

# Iterate over all the values of each column and extract their dtypes
for col in cc_apps.columns:
    # Compare if the dtype is object
    if cc_apps[col].dtypes=='object':
    # Use LabelEncoder to do the numeric transformation
        cc_apps[col]=le.fit_transform(cc_apps[col])
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    int64  
 1   1       690 non-null    int64  
 2   2       690 non-null    float64
 3   3       690 non-null    int64  
 4   4       690 non-null    int64  
 5   5       690 non-null    int64  
 6   6       690 non-null    int64  
 7   7       690 non-null    float64
 8   8       690 non-null    int64  
 9   9       690 non-null    int64  
 10  10      690 non-null    int64  
 11  11      690 non-null    int64  
 12  12      690 non-null    int64  
 13  13      690 non-null    int64  
 14  14      690 non-null    int64  
 15  15      690 non-null    int64  
dtypes: float64(2), int64(14)
memory usage: 86.4 KB


In [ ]:
## 7. Dividir el conjunto de datos en conjuntos de prueba y de entrenamiento

Hemos convertido con éxito todos los valores no numéricos en valores numéricos. Ahora, dividiremos nuestros datos en conjuntos de entrenamientos y conjuntos de pruebas para preparar nuestros datos para dos fases diferentes del modelado de aprendizaje automático: Entrenamiento y pruebas. 

Idealmente, no se debe usar información de los datos de prueba para escalar los datos de entrenamiento o se debe usar para dirigir el proceso de entrenamiento de un modelo de aprendizaje automático. Por lo tanto, primero dividimos los datos y luego aplicamos la escala. Además, funciones como DriversLicense y ZipCode no son tan importantes como las otras funciones del conjunto de datos para predecir las aprobaciones de tarjetas de crédito. Deberíamos descartarlos para diseñar nuestro modelo de aprendizaje automático con el mejor conjunto de características. En la literatura sobre ciencia de datos, esto a menudo se denomina *selección de características*.


In [25]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Drop the features 11 and 13 
cc_apps = cc_apps.drop([11, 13], axis=1)

#verify
print(cc_apps.head())

print("\n")

#and convert the DataFrame to a NumPy array
cc_apps = cc_apps.values

# Segregate features and labels into separate variables
X,y = cc_apps[:,0:13] , cc_apps[:,13]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.33, random_state=42)

   0    1      2   3   4   5   6     7   8   9   10  12   14  15
0   1  156  0.000   2   1  13   8  1.25   1   1   1   0    0   0
1   0  328  4.460   2   1  11   4  3.04   1   1   6   0  560   0
2   0   89  0.500   2   1  11   4  1.50   1   0   0   0  824   0
3   1  125  1.540   2   1  13   8  3.75   1   1   5   0    3   0
4   1   43  5.625   2   1  13   8  1.71   1   0   0   2    0   0




## 8. Procesamiento previo de los datos (Parte 2)

Los datos ahora se dividen en dos conjuntos separados: conjuntos de entrenamiento y de prueba, respectivamente. Solo nos queda un último paso de preprocesamiento de escala antes de que podamos ajustar un modelo de aprendizaje automático a los datos.

Ahora, intentemos comprender qué significan estos valores escalados en el mundo real. Usemos CreditScore como ejemplo. La calificación crediticia de una persona es su solvencia basada en su historial crediticio. Cuanto mayor sea este número, más confiable financieramente se considera una persona. Por lo tanto, un CreditScore de 1 es el más alto, ya que estamos reescalando todos los valores al rango de 0-1.

In [27]:
# Import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
# Instantiate MinMaxScaler and use it to rescale X_train and X_test
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.fit_transform(X_test)

## 9. Adaptación de un modelo de regresión logística al entrenamiento

Esencialmente, predecir si una solicitud de tarjeta de crédito será aprobada o no es una tarea de clasificación. Según [UCI](https://archive.ics.uci.edu/ml/datasets/Credit+Approval), nuestro conjunto de datos contiene más instancias que corresponden al estado "Denegado" que instancias correspondientes al estado "Aprobado". Específicamente, de 690 casos, hay 383 (55.5%) solicitudes que fueron denegadas y 307 (44.5%) solicitudes que fueron aprobadas.

Esto nos da un punto de referencia. Un buen modelo de aprendizaje automático debería poder predecir con precisión el estado de las aplicaciones con respecto a estas estadísticas.

¿Qué modelo deberíamos elegir? Una pregunta que debe hacerse es: ¿las características que afectan el proceso de decisión de aprobación de la tarjeta de crédito están correlacionadas entre sí? Aunque podemos medir la correlación, eso está fuera del alcance de este notebook, por lo que confiaremos en nuestra intuición de que de hecho están correlacionados por ahora. Debido a esta correlación, aprovecharemos el hecho de que los modelos lineales generalizados funcionan bien en estos casos. Comencemos nuestro modelado de aprendizaje automático con un modelo de regresión logística (un modelo lineal generalizado).


In [ ]:
# Import LogisticRegression
from sklearn.linear_model import LogisticRegression

# Instantiate a LogisticRegression classifier with default parameter values
logreg = LogisticRegression()
# Fit logreg to the train set
logreg.fit(rescaledX_train,y_train)

## 10. Hacer predicciones y evaluar el desempeño

Pero, ¿qué tan bien funciona nuestro modelo?

Ahora evaluaremos nuestro modelo en el conjunto de prueba con respecto a la precisión de clasificación. Pero también echaremos un vistazo a la matriz de confusión del modelo. En el caso de predecir aplicaciones de tarjetas de crédito, es igualmente importante ver si nuestro modelo de aprendizaje automático es capaz de predecir el estado de aprobación de las aplicaciones como denegadas que originalmente fueron denegadas. Si nuestro modelo no está funcionando bien en este aspecto, entonces podría terminar aprobando la aplicación que debería haber sido aprobada. La matriz de confusión nos ayuda a ver el desempeño de nuestro modelo desde estos aspectos.


In [29]:
# Import confusion_matrix
from sklearn.metrics import confusion_matrix

# Use logreg to predict instances from the test set and store it
y_pred = logreg.predict(rescaledX_test)

# Get the accuracy score of logreg model and print it
print("Accuracy of logistic regression classifier: ", logreg.score(rescaledX_test, y_test))

# Print the confusion matrix of the logreg model
print("Confusion matrix of regression classifier: ", confusion_matrix(y_test, y_pred))

Accuracy of logistic regression classifier:  0.8377192982456141
Confusion matrix of regression classifier:  [[92 11]
 [26 99]]


In [ ]:
## 11. Búsqueda de cuadrículas y mejora del rendimiento del modelo

Nuestro modelo era bastante bueno. Pudo producir una puntuación de precisión de casi el 84%.

Para la matriz de confusión, el primer elemento de la primera fila de la matriz de confusión denota los verdaderos negativos, es decir, el número de instancias negativas (aplicaciones denegadas) predichas correctamente por el modelo. Y el último elemento de la segunda fila de la matriz de confusión denota los verdaderos positivos, es decir, el número de instancias positivas (aplicaciones aprobadas) predichas correctamente por el modelo.

Veamos si podemos hacerlo mejor. Podemos realizar una búsqueda en cuadrícula de los parámetros del modelo para mejorar la capacidad del modelo para predecir las aprobaciones de tarjetas de crédito.

La implementación de scikit-learn de la regresión logística consta de diferentes hiperparámetros, pero buscaremos en la cuadrícula los dos siguientes:

- tol
- max_iter

In [34]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV
# Define the grid of values for tol and max_iter
tol = [0.01,0.001,0.0001]
max_iter = [100,150,200]
# Create a dictionary where tol and max_iter are keys and the lists of their values are corresponding values
param_grid = dict(tol=tol, max_iter=max_iter)

In [32]:
## 12. Encontrar el modelo con mejor rendimiento

Hemos definido la cuadrícula de valores de hiperparámetros y los hemos convertido en un formato de diccionario único que GridSearchCV() espera como uno de sus parámetros. Ahora, comenzaremos la búsqueda en la cuadrícula para ver qué valores funcionan mejor.

Crearemos una instancia de GridSearchCV() con nuestro modelo de registro anterior con todos los datos que tenemos. En lugar de pasar los conjuntos de entrenamient y de prueba por separado, proporcionaremos *X* (versión escalada) e *y*. También indicaremos a GridSearchCV() que realice una validación cruzada de cinco pliegues.

Terminaremos el cuaderno almacenando la puntuación mejor lograda y los mejores parámetros respectivos.


In [36]:
# Instantiate GridSearchCV with the required parameters
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

# Use scaler to rescale X and assign it to rescaledX
rescaledX = scaler.fit_transform(X)

# Fit data to grid_model
grid_model_result = grid_model.fit(rescaledX, y)

# Summarize results
best_score, best_params = grid_model_result.best_score_,grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

Best: 0.850725 using {'max_iter': 100, 'tol': 0.01}
